<a href="https://colab.research.google.com/github/diegoalvzfdez/master-data-science/blob/master/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

#Obtenemos los datos guardados en la nube de Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/My\ Drive/TFM\ Data\ Science/Gonzalo\ Velarde

/content/drive/My Drive/TFM Data Science/Gonzalo Velarde


In [0]:
#leemos el fichero
df = pd.read_csv('Velarde Rodriguez_Gonzalo_OD_31052017_120629_ELE.CSV', sep = ';', index_col= 0, header = 0, encoding = "ISO-8859-1")

Tenemos un fichero con la siguiente forma:

![](https://drive.google.com/uc?export=view&id=15UODzawnIyQqmrKIcTLwNOBOkcjqJh28)

Lo que tenemos es una representación de la profundidad de un ojo (eje z) en las coordenadas x (primera columna) y coordenadas y (primera fila)

Para poder manejar estos datos, en primer lugar buscaremos una manera de poder tener los datos más significativos de la manera más comoda para introducirlos en un clasificador.

En esta zona apuntaré las ideas que se me vayan ocurriendo para tratar dicha problemática (esperemos que no tener que escribir muchas):

1. Conseguir un vector de los n puntos con mayor profundidad: para ello vamos a tener que stackear los datos para conseguir todos los valores en una misma columna para después ordenarlos.

# Idea 1

In [0]:
#Stackeamos los datos para obtener las coordenadas de cada punto
df_stacked = df.stack()
df_stacked = df_stacked.reset_index()
df_stacked = df_stacked.rename(index = str, columns = {'FRONT': 'X', 'level_1': 'Y', 0: 'Z'})
#necesitamos una columna que simule el índice del dataframe
df_stacked['ID'] = df_stacked.index
df_stacked['ID'] = pd.to_numeric(df_stacked['ID'])
df_stacked.head()


,X,Y,Z,ID
0,"5,200","-0,900",1967,0
1,"5,200","-0,800",1954,1
2,"5,200","-0,700",1943,2
3,"5,200","-0,600",1933,3
4,"5,200","-0,500",1925,4


Como se puede observar en el fichero, las coordenadas del ojo finalizan cuando en lugar de un numero hay un nombre en la columna del eje X, por lo que a partir de ese punto hay que cortar todo el dataset

In [0]:
df_stacked_first_eye = df_stacked[df_stacked['ID'] < list(df_stacked[df_stacked['X'] == 'BACK']['ID'])[0]]
df_stacked_first_eye.tail()

,X,Y,Z,ID
9973,"-5,900","0,100",2650,9973
9974,"-5,900","0,200",2653,9974
9975,"-5,900","0,300",2658,9975
9976,"-5,900","0,400",2665,9976
9977,"-5,900","0,500",2674,9977


Para el siguiente paso necesitamos obtener los puntos de mayor profundidad, para obtener los puntos más característicos de nuestra muestra.

In [0]:
#Borramos las columnas que no nos son necesarias
df_stacked_first_eye.drop(['X', 'Y', 'ID'], axis = 1, inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [0]:
#ordenamos los valores de Z para obtener los puntos más elevados
df_stacked_first_eye['Z'] = pd.to_numeric(df_stacked_first_eye['Z'])
df_stacked_first_eye.sort_values('Z', ascending=False, inplace = True)
df_stacked_first_eye.head(15)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Z
9965,2687
9940,2681
9907,2678
9966,2677
9977,2674
9868,2671
9967,2667
9776,2667
9976,2665
9666,2664
